# Distracted Driver Classification

### Satchel Grant

The goal of this notebook is to classify the distraction type of the driver of a car. For my personal purposes, I wanted to try implementing an inception net for image classification.

## Initial Imports

In [1]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle

def show_img(img):
    plt.imshow(img)
    plt.show()

## Read in Data

## Preprocessing

In [ ]:
def centernorm(images, mean, stddev):
    return (images-mean)/stddev

def one_hot_encode(labels, n_labels):
    encoded_labels = np.zeros((labels.shape[0], n_labels), dtype=np.float32)
    for i,label in enumerate(labels):
        encoded_labels[i,int(label)] = 1
    return encoded_labels

## Define Neural Net Architecture
The inception net is a way to use multiple types of convolution structures together. The idea is to run each a 1x1, 3x3, and 5x5 convolution kernel in addition to a max-pool operation at each layer in the net.

## Run Session